# Homework 13 - Network Compression

Author: Liang-Hsuan Tseng (b07502072@ntu.edu.tw), modified from ML2021-HW13

If you have any questions, feel free to ask: ntu-ml-2022spring-ta@googlegroups.com

[**Link to HW13 Slides**](https://docs.google.com/presentation/d/1nCT9XrInF21B4qQAWuODy5sonKDnpGhjtcAwqa75mVU/edit#slide=id.p)

### Outline

* [Packages](#Packages) - intall some required packages.
* [Dataset](#Dataset) - something you need to know about the dataset.
* [Configs](#Configs) - the configs of the experiments, you can change some hyperparameters here.
* [Architecture_Design](#Architecture_Design) - depthwise and pointwise convolution examples and some useful links.
* [Knowledge_Distillation](#Knowledge_Distillation) - KL divergence loss for knowledge distillation and some useful links.
* [Training](#Training) - training loop implementation modified from HW3.
* [Inference](#Inference) - create submission.csv by loading the best.ckpt from the .



### Packages
First, we need to import some useful packages. Since the `torchsummary` package is not installed by default, we have to install it via `pip` manually.

In [1]:
# !pip install torchsummary
# !wget https://github.com/andybi7676/ml2022spring-hw13/raw/main/food11-hw13.tar.gz
# !tar -xzf ./food11-hw13.tar.gz # Could take some time


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# Import some useful packages for this homework
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset # "ConcatDataset" and "Subset" are possibly useful.
from torchvision.datasets import DatasetFolder, VisionDataset
from torchsummary import summary
from tqdm import tqdm
import random

### Configs
In this part, you can specify some variables and hyperparameters as your configs.

In [3]:
cfg = {
    'dataset_root': './food11-hw13',
    'save_dir': './outputs',
    'exp_name': "rkd_mobilenet_randugmentation_mixup",
    'batch_size': 128,
    'lr': 3e-3,
    'seed': 64,
    'loss_fn_type': 'RKD', # simple baseline: CE, medium baseline: KD. See the Knowledge_Distillation part for more information.
    'weight_decay': 1e-5,
    'grad_norm_max': 10,
    'n_epochs': 500,
    'patience': 100,
    'augmentation': 'rand', # or rand,
    "mixup_prob": 0.5
}

In [4]:
myseed = cfg['seed']  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
random.seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

save_path = os.path.join(cfg['save_dir'], cfg['exp_name']) # create saving directory
os.makedirs(save_path, exist_ok=True)

# define simple logging functionality
log_fw = open(f"{save_path}/log.txt", 'w') # open log file to save log outputs
def log(text):     # define a logging function to trace the training process
    print(text)
    log_fw.write(str(text)+'\n')
    log_fw.flush()

log(cfg)  # log your configs to the log file

{'dataset_root': './food11-hw13', 'save_dir': './outputs', 'exp_name': 'rkd_mobilenet_randugmentation_mixup', 'batch_size': 128, 'lr': 0.003, 'seed': 64, 'loss_fn_type': 'RKD', 'weight_decay': 1e-05, 'grad_norm_max': 10, 'n_epochs': 500, 'patience': 100, 'augmentation': 'rand', 'mixup_prob': 0.5}


### Dataset
We use Food11 dataset for this homework, which is similar to homework3. But remember, Please DO NOT utilize the dataset of HW3. We've modified the dataset, so you should only access the dataset by loading it in this kaggle notebook or through the links provided in the HW13 colab notebooks.

In [5]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    if len(filenames) > 0:
        print(f"{dirname}: {len(filenames)} files.") # Show the .jpg file amounts in each split.

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Next, specify train/valid transform for image data augmentation.
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms. You can also apply the knowledge or experience you learned in HW3.

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# define training/testing transforms
test_tfm = transforms.Compose([
    # It is not encouraged to modify this part if you are using the provided teacher model. This transform is stardard and good enough for testing.
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])
if cfg['augmentation'] == 'default':
    train_tfm = transforms.Compose([
        # add some useful transform or augmentation here, according to your experience in HW3.
        transforms.Resize(256),  # You can change this
        transforms.CenterCrop(224), # You can change this, but be aware of that the given teacher model's input size is 224.
        # The training input size of the provided teacher model is (3, 224, 224).
        # Thus, Input size other then 224 might hurt the performance. please be careful.
        transforms.ToTensor(),
        normalize,
    ])
elif cfg['augmentation'] in ['rand', 'auto']:
    train_tfm = transforms.Compose([
        # add some useful transform or augmentation here, according to your experience in HW3.
        transforms.Resize([224,224]),  # You can change this
        transforms.RandAugment() if cfg['augmentation'] == 'rand' else transforms.AutoAugment(),
        transforms.ToTensor(),
        normalize,
    ])

In [7]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files = None):
        super().__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [8]:
train_set = FoodDataset(os.path.join(cfg['dataset_root'],"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=10, pin_memory=True)

valid_set = FoodDataset(os.path.join(cfg['dataset_root'], "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=10, pin_memory=True)

One ./food11-hw13/training sample ./food11-hw13/training/0_0.jpg
One ./food11-hw13/validation sample ./food11-hw13/validation/0_0.jpg


### Architecture_Design

In this homework, you have to design a smaller network and make it perform well. Apparently, a well-designed architecture is crucial for such task. Here, we introduce depthwise and pointwise convolutions. They are some common techniques for architecture design when it comes to network compression.

<img src="https://i.imgur.com/LFDKHOp.png" width=400px>

* explanation of depthwise and pointwise convolutions:
    * [prof. Hung-yi Lee's slides(p.24~p.30, especially p.28)](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/tiny_v7.pdf)

In [9]:
import math

In [10]:
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, n_class=11, input_size=128, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 100
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 1, 2],
            [6, 32, 1, 2],
            [6, 64, 2, 2],
        ]

        # building first layer
        assert input_size % 32 == 0
        # input_channel = make_divisible(input_channel * width_mult)  # first channel is always 32!
        self.last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = make_divisible(c * width_mult) if t > 1 else c
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Linear(self.last_channel, n_class)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


* other useful techniques and popular network architecture designs for network compression
    * [group convolution](https://www.researchgate.net/figure/The-transformations-within-a-layer-in-DenseNets-left-and-CondenseNets-at-training-time_fig2_321325862) (Actually, depthwise convolution is a specific type of group convolution)
    * [SqueezeNet](!https://arxiv.org/abs/1602.07360)
    * [MobileNet](!https://arxiv.org/abs/1704.04861)
    * [ShuffleNet](!https://arxiv.org/abs/1707.01083)
    * [Xception](!https://arxiv.org/abs/1610.02357)
    * [GhostNet](!https://arxiv.org/abs/1911.11907)


Now, let's define the **student network architecture**. Here, we have a very simple network formed by some convolution layers and pooling layers. You can replace the regular convolution layers with the depthwise and pointwise convolutions. In this way, you can further increase the depth or the width of your network architecture.

In [11]:
# Define your student network here. You have to copy-paste this code block to HW13 GradeScope before deadline.
# We will use your student network definition you submit to evaluate your results(including the total parameter amount).
def create_deepwise_point_wise_conv(nin, nout, kernel_size=3, stride=1, padding=0):
     return nn.Sequential(nn.Conv2d(nin, nin, kernel_size=kernel_size, stride=stride, padding=padding, groups=nin), 
                                   nn.BatchNorm2d(nin),
                                  nn.LeakyReLU(),
                                  nn.Conv2d(nin, nout, kernel_size=1),
                                  nn.MaxPool2d(2)
                                 )

def create_cnn(nin, nout):
     return nn.Sequential(nn.Conv2d(nin, nout, 3), 
                            nn.BatchNorm2d(nout),
                            nn.LeakyReLU())
    
class StudentNet(nn.Module):
    def __init__(self):
      super().__init__()

      # ---------- TODO ----------
      # Modify your model architecture

      self.cnn = nn.Sequential(
        create_cnn(3, 32),        
        create_cnn(32, 64),
        create_deepwise_point_wise_conv(3, 8),
        create_deepwise_point_wise_conv(8, 16),
        create_deepwise_point_wise_conv(16, 32),
        # create_deepwise_point_wise_conv(32, 64),
        create_deepwise_point_wise_conv(32, 64),
        create_deepwise_point_wise_conv(64, 128),
        # create_deepwise_point_wise_conv(64, 128),
        create_deepwise_point_wise_conv(128, 256),
        nn.LeakyReLU(),
        # Here we adopt Global Average Pooling for various input size.
        nn.AdaptiveAvgPool2d((1, 1)),
      )
      self.fc = nn.Sequential(
        nn.LeakyReLU(),
        nn.Linear(256, 11),
      )
      
    def forward(self, x):
      out = self.cnn(x)
      out = out.view(out.size()[0], -1)
      return self.fc(out)

def get_student_model():
    model = MobileNetV2(width_mult=1)
    return model

# End of definition of your student model and the get_student_model API
# Please copy-paste the whole code block, including the get_student_model function.
student_model = get_student_model()
summary(student_model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

After specifying the student network architecture, please use `torchsummary` package to get information about the network and verify the total number of parameters. Note that your student network should not exceed the limit (`Total params` in `torchsummary` < 10,000). 

In [12]:
# This is how we will evaluate your student model.
# Do not modify this code block.
student_model = get_student_model()
# summary(student_model, (3, 224, 224), device='cpu')

In [13]:
# Load provided teacher model (model architecture: resnet18, num_classes=11, test-acc ~= 89.9%)
teacher_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=11)
# load state dict
teacher_ckpt_path = os.path.join(cfg['dataset_root'], "resnet18_teacher.ckpt")
teacher_model.load_state_dict(torch.load(teacher_ckpt_path, map_location='cuda'))
# Now you already know the teacher model's architecture. You can take advantage of it if you want to pass the strong or boss baseline. 
# Source code of resnet in pytorch: (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py)
# You can also see the summary of teacher model. There are 11,182,155 parameters totally in the teacher model
# summary(teacher_model, (3, 224, 224), device='cpu')

Using cache found in /home/lpz106u/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

### Knowledge_Distillation

<img src="https://i.imgur.com/H2aF7Rv.png=100x" width="400px">

Since we have a learned big model, let it teach the other small model. In implementation, let the training target be the prediction of big model instead of the ground truth.

**Why it works?**
* If the data is not clean, then the prediction of big model could ignore the noise of the data with wrong labeled.
* There might have some relations between classes, so soft labels from teacher model might be useful. For example, Number 8 is more similar to 6, 9, 0 than 1, 7.


**How to implement?**
* $Loss = \alpha T^2 \times KL(p || q) + (1-\alpha)(\text{Original Cross Entropy Loss}), \text{where } p=softmax(\frac{\text{student's logits}}{T}), \text{and } q=softmax(\frac{\text{teacher's logits}}{T})$
* very useful link: [pytorch docs of KLDivLoss with examples](!https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html)
* original paper: [Distilling the Knowledge in a Neural Network](!https://arxiv.org/abs/1503.02531)

In [14]:
# Implement the loss function with KL divergence loss for knowledge distillation.
# You also have to copy-paste this whole block to HW13 GradeScope. 
import torch.nn.functional as F


def loss_fn_kd(student_logits, labels, teacher_logits, alpha=0.5, temperature=5.):
    # ------------TODO-------------
    # Refer to the above formula and finish the loss function for knowkedge distillation using KL divergence loss and CE loss.
    # If you have no idea, please take a look at the provided useful link above.
    p = torch.log_softmax(student_logits / temperature, dim=-1)    
    q = torch.softmax(teacher_logits / temperature, dim=-1)
    return alpha * (temperature ** 2) * F.kl_div(p, q, reduction='batchmean') + (1 - alpha) * F.cross_entropy(student_logits, labels)


'''
ref: https://github.com/lenscloth/RKD/blob/master/metric/loss.py
'''

class RKD(nn.Module):
    '''
	Relational Knowledge Distillation
	https://arxiv.org/pdf/1904.05068.pdf
	'''

    def __init__(self, w_dist, w_angle, alpha=0.5):
        super(RKD, self).__init__()

        self.w_dist = w_dist
        self.w_angle = w_angle
        
        self.alpha = alpha

    def forward(self, feat_s, labels, feat_t):
        loss = self.w_dist * self.rkd_dist(feat_s, feat_t) + \
            self.w_angle * self.rkd_angle(feat_s, feat_t)
        loss = self.alpha * loss +  (1 - self.alpha) * F.cross_entropy(feat_s, labels)

        return loss
    
    def rkd_dist(self, feat_s, feat_t):
      feat_t_dist = self.pdist(feat_t, squared=False)
      mean_feat_t_dist = feat_t_dist[feat_t_dist>0].mean()
      feat_t_dist = feat_t_dist / mean_feat_t_dist

      feat_s_dist = self.pdist(feat_s, squared=False)
      mean_feat_s_dist = feat_s_dist[feat_s_dist>0].mean()
      feat_s_dist = feat_s_dist / mean_feat_s_dist

      loss = F.smooth_l1_loss(feat_s_dist, feat_t_dist)

      return loss


    def rkd_angle(self, feat_s, feat_t):
        # N x C --> N x N x C
        feat_t_vd = (feat_t.unsqueeze(0) - feat_t.unsqueeze(1))
        norm_feat_t_vd = F.normalize(feat_t_vd, p=2, dim=2)
        feat_t_angle = torch.bmm(norm_feat_t_vd,
                                 norm_feat_t_vd.transpose(1, 2)).view(-1)

        feat_s_vd = (feat_s.unsqueeze(0) - feat_s.unsqueeze(1))
        norm_feat_s_vd = F.normalize(feat_s_vd, p=2, dim=2)
        feat_s_angle = torch.bmm(norm_feat_s_vd,
                                 norm_feat_s_vd.transpose(1, 2)).view(-1)

        loss = F.smooth_l1_loss(feat_s_angle, feat_t_angle)

        return loss

    def pdist(self, feat, squared=False, eps=1e-12):
        feat_square = feat.pow(2).sum(dim=1)
        feat_prod = torch.mm(feat, feat.t())
        feat_dist = (feat_square.unsqueeze(0) + feat_square.unsqueeze(1) -
                     2 * feat_prod).clamp(min=eps)

        if not squared:
            feat_dist = feat_dist.sqrt()

        feat_dist = feat_dist.clone()
        feat_dist[range(len(feat)), range(len(feat))] = 0

        return feat_dist


In [15]:
# choose the loss function by the config
if cfg['loss_fn_type'] == 'CE':
    # For the classification task, we use cross-entropy as the default loss function.
    loss_fn = nn.CrossEntropyLoss() # loss function for simple baseline.

elif cfg['loss_fn_type'] == 'KD': # KD stands for knowledge distillation
    loss_fn = loss_fn_kd # implement loss_fn_kd for the report question and the medium baseline.
elif cfg['loss_fn_type'] =='RKD':
    loss_fn = RKD(25, 50)

# You can also other types of knowledge distillation techniques, but use function name other than `loss_fn_kd`
# For example:
# def loss_fn_custom_kd():
#     pass
# if cfg['loss_fn_type'] == 'custom_kd':
#     loss_fn = loss_fn_custom_kd

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
log(f"device: {device}")
teacher_model.to(device)
teacher_model.eval()
# The number of training epochs and patience.
n_epochs = cfg['n_epochs']
patience = cfg['patience'] # If no improvement in 'patience' epochs, early stop

device: cuda


### Training
implement training loop for simple baseline, feel free to modify it.

In [16]:
import wandb
import numpy as np


def join_cfg(cfg:dict):
    ignore = ['dataset_root', 'save_dir', 'exp_name', 'batch_size', 'seed']
    result = []
    for k, v in cfg.items():
        if k not in ignore:
            result.append(f'{k}-{v}')
    return '_'.join(result)
    
# Initialize a model, and put it on the device specified.
student_model.to(device)
# teacher_model.to(device) # MEDIUM BASELINE

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(student_model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay']) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.5, verbose=False)
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0.0
wandb.init(project='ML_hw13', config=cfg, name=join_cfg(cfg))
teacher_model.eval()  # MEDIUM BASELINE
pbar = tqdm(range(n_epochs))
for epoch in pbar:

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    student_model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    train_lens = []

    for batch in tqdm(train_loader, leave=False):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)
        #print(imgs.shape,labels.shape)
        
        # mixup
        if random.random() < cfg['mixup_prob']:
            indices = torch.randperm(imgs.size(0), device=imgs.device, dtype=torch.long)
            gamma = np.random.beta(0.2, 0.2)
            labels = F.one_hot(labels, 11)
            imgs = imgs.mul(gamma).add(imgs[indices], alpha=1-gamma)
            labels = labels.mul(gamma).add(labels[indices], alpha=1-gamma)


        # Forward the data. (Make sure data and model are on the same device.)
        with torch.no_grad():  # MEDIUM BASELINE
            teacher_logits = teacher_model(imgs)  # MEDIUM BASELINE
        
        logits = student_model(imgs)

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = loss_fn(logits, labels, teacher_logits) # MEDIUM BASELINE
#         loss = loss_fn(logits, labels) # SIMPLE BASELINE
        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=cfg['grad_norm_max'])

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        # acc = (logits.argmax(dim=-1) == labels).float().sum().item()

        # Record the loss and accuracy.
        train_batch_len = len(imgs)
        train_loss.append(loss.item() * train_batch_len)
        # train_accs.append(acc)
        train_lens.append(train_batch_len)
    scheduler.step()
    train_loss = sum(train_loss) / sum(train_lens)
    # train_acc = sum(train_accs) / sum(train_lens)
    train_metrics = {'train/loss': train_loss}
    wandb.log(train_metrics, commit=False)

    # Print the information.
    # log(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    pbar.set_postfix(train_metrics)
    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    student_model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []
    valid_lens = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader, leave=False):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        imgs = imgs.to(device)
        labels = labels.to(device)

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = student_model(imgs)
            teacher_logits = teacher_model(imgs) # MEDIUM BASELINE

#         We can still compute the loss (but not the gradient).
        loss = loss_fn(logits, labels, teacher_logits) # MEDIUM BASELINE
#         loss = loss_fn(logits, labels) # SIMPLE BASELINE

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels).float().sum().item()

        # Record the loss and accuracy.
        batch_len = len(imgs)
        valid_loss.append(loss.item() * batch_len)
        valid_accs.append(acc)
        valid_lens.append(batch_len)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / sum(valid_lens)
    valid_acc = sum(valid_accs) / sum(valid_lens)

    # update logs
    
    # if valid_acc > best_acc:
    #     log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    # else:
    #     log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    val_metrics = {'val/loss': valid_loss, 'val/acc': valid_acc, 'val/best_acc': best_acc}
    wandb.log(val_metrics, commit=True)
    pbar.set_postfix(val_metrics)


    # save models
    if valid_acc > best_acc:
        # log(f"Best model found at epoch {epoch}, saving model")
        torch.save(student_model.state_dict(), f"{save_path}/student_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            log(f"No improvment {patience} consecutive epochs, early stopping")
            break
log("Finish training") 
log_fw.close()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tedli (use `wandb login --relogin` to force relogin)


 78%|███████▊  | 388/500 [2:19:50<40:22, 21.63s/it, val/loss=1.82, val/acc=0.807, val/best_acc=0.812, _timestamp=1.65e+9, _runtime=8397]  

No improvment 100 consecutive epochs, early stopping
Finish training


### Inference
load the best model of the experiment and generate submission.csv

In [17]:
# Load model from {exp_name}/student_best.ckpt
student_model_best = get_student_model() # get a new student model to avoid reference before assignment.
ckpt_path = f"{save_path}/student_best.ckpt" # the ckpt path of the best student model.
student_model_best.load_state_dict(torch.load(ckpt_path, map_location='cpu')) # load the state dict and set it to the student model
student_model_best.to(device) # set the student model to device
ensemble_num = 30
# Start evaluate
student_model_best.eval()
eval_preds = [] # storing predictions of the evaluation dataset
eval_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=train_tfm)

def pad4(i):
    return "0"*(4-len(str(i))) + str(i)
ids = [pad4(i) for i in range(0,len(eval_set))]

# Iterate the validation set by batches.
for i in tqdm(range(len(eval_set))):
    imgs = []
    for _ in range(ensemble_num):
        imgs.append(eval_set[i][0])
    imgs = torch.stack(imgs)
    with torch.no_grad():
        logits = student_model_best(imgs.to(device))
        pred = logits.sum(0).argmax(-1).item()
    eval_preds.append(pred)
df = pd.DataFrame({'Id': ids, 'Category': eval_preds})
df.to_csv(f"{save_path}/submission_aug.csv", index=False) # now you can download the submission.csv and upload it to the kaggle competition.


# eval_preds = [] # storing predictions of the evaluation dataset
# eval_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=train_tfm)
# eval_loader = DataLoader(eval_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=10, pin_memory=True)
# for batch in tqdm(eval_loader):
    
#     # A batch consists of image data and corresponding labels.
#     imgs, _ = batch
#     # We don't need gradient in evaluation.
#     # Using torch.no_grad() accelerates the forward process.
#     with torch.no_grad():
#         logits = student_model_best(imgs.to(device))
#         preds = list(logits.argmax(dim=-1).squeeze().cpu().numpy())
#     # loss and acc can not be calculated because we do not have the true labels of the evaluation set.
#     eval_preds += preds
# df = pd.DataFrame({'Id': ids, 'Category': eval_preds})
# df.to_csv(f"{save_path}/submission_original.csv", index=False) # now you can download the submission.csv and upload it to the kaggle competition.

# Save prediction results



One ./food11-hw13/evaluation sample ./food11-hw13/evaluation/0000.jpg


100%|██████████| 3347/3347 [16:52<00:00,  3.31it/s]  


One ./food11-hw13/evaluation sample ./food11-hw13/evaluation/0000.jpg


100%|██████████| 27/27 [00:05<00:00,  4.64it/s]


> Don't forget to answer the report questions on GradeScope!